Exercise : Container images and layers
------------------------------------------------------

We want to find the Apache Web Server Image on docker.io.

We pull this image from various registries onto our local machine and analyze the content.


In [ ]:
! docker search apache

Unfortunately, `docker search` does not work with other registries, so we have to search there via the web interface.

Andere Ã¶ffentliche Registries sind u.a.
* [AWS Public Registry](https://public.ecr.aws)
* [RedHat](https://quay.io)

If a [GitLab Umgebung](https://about.gitlab.com/) is available or the public one is used, the container images can be placed in the same repository as the source code.

* [https://gitlab.com/mc-b/misegr/container_registry](https://gitlab.com/mc-b/misegr/container_registry)

- - -

### Container Images holen

To get around the Docker "pull rate limits", we don't pull the Apache Web Server from `docker.io` but from the AWS Public Registry. There we find, with the prefix `docker/library`, practically all "Docker Official Images".

In [ ]:
! docker pull public.ecr.aws/docker/library/httpd
! docker image ls

- - -

### Save container image in `tar` format

An image registry is not always available.

Alternatively, container images can also be saved and loaded from/to the [tar](https://wiki.ubuntuusers.de/tar/) format

In [ ]:
! docker save public.ecr.aws/docker/library/httpd -o httpd.tar
! ls -ls *.tar

The file can then be transported to our target environment, e.g. via a USB stick, and imported again there.

To demonstrate this, we first delete the cached image and then import our [tar](https://wiki.ubuntuusers.de/tar/) export.

In [ ]:
%%bash
docker rmi public.ecr.aws/docker/library/httpd
docker image ls
echo "--------------------"
docker load -i httpd.tar
docker image ls

The created [tar](https://wiki.ubuntuusers.de/tar/) file can be extracted and the created layers can be analyzed.

In [ ]:
! tar xvf httpd.tar
! cat manifest.json | jq

Additional `tar xvf <uuid>/layer.tar` and `cat <uuid>/json provide more information about the image.

Try it out by replacing `[uuid]` with a UUID from above:

In [ ]:
! tar xvf [uuid]/layer.tar
! cat [uuid]/json | jq

- - -

### Unpack container file system

From time to time, you just want to explore the container's filesystem without starting the container itself.

This works with `docker create` instead of `run` and exporting the file system:

In [ ]:
! mkdir apache
! docker export $(docker create public.ecr.aws/docker/library/httpd) | tar -C apache -xvf -

This created an `apache` directory and extracted the contents of the container image from there.

Using `cat` or `ls -l apache` we can analyze the content

In [ ]:
! cat apache/etc/issue
! echo "-----httpd.conf -------"
! cat apache/usr/local/apache2/conf/httpd.conf

***
Questions
======

Answer the questions individually or in groups and compare them with the answers

What is the difference between a Docker Registry and a repository
<details><summary>Answer</summary>
The container images are stored in the Docker Registry. A repository stores different versions of images per container image.
</p></details>

---

What is Docker Hub?
<details><summary>Answer</summary>
A container registry where container images are stored. Docker Hub is provided by the Docker company.
</p></details>

---

What are the alternatives?
<details><summary>Answer</summary>
Virtually every cloud provider provides a container registry. Providers for managing build artifacts (e.g. Sonatype Nexus) also provide Docker registries or have their functionality integrated.
</p></details>

---

Why should the company use its own container registry?
<details><summary>Answer</summary>
Security, or the possible lack thereof. It cannot be guaranteed that all container images on Docker Hub are secure.
</p></details>
